## 장식자 (Decorators)

In [1]:
def base(시작값):
    i = 0
    fn = lambda x, y: x + y + 시작값  # 익명함수
    return fn

In [5]:
def base(시작값):
    i = 0
    def fn(x, y):  # 익명함수 대신, 함수를 직정 정의하셔도 됩니다.
        return x + y + 시작값
    return fn

In [8]:
# 아래와 같이 만들 필요없이, base 함수를 활용하세요.
# def base_10(x, y):
#     return x + y + 10
# def base_20(x, y):
#     return x + y + 20
# def base_30(x, y):
#     return x + y + 30

base_10 = base(10)
base_20 = base(20)
base_30 = base(30)

In [4]:
def mysum1(x, y):
    return x + y + 10

mysum2 = lambda x, y: x + y + 10

print(mysum1(1, 2))
print(mysum2(1, 2))

13
13
